In [11]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
import cv2
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.layers import Lambda
from PIL import Image
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical


In [12]:
class CustomGenerator(keras.utils.Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size

    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]
        return np.array([
            (cv2.imread(str(file_name)))
            for file_name in batch_x]), np.array(batch_y)

In [13]:
path = Path("test_data_smaller")
df = pd.read_csv(path/'artists.csv')

#check distribution of paintings
artists_df = df[['name', 'paintings']].groupby(['name'], as_index = False).sum()
names = artists_df.sort_values('paintings', ascending = False)[:50]
names

,name,paintings
48,Vincent van Gogh,877
10,Edgar Degas,702
33,Pablo Picasso,439
38,Pierre-Auguste Renoir,336
0,Albrecht Dürer,328
35,Paul Gauguin,311
15,Francisco Goya,291
42,Rembrandt,262
1,Alfred Sisley,259
46,Titian,255


In [14]:
#remove spaces from names
images_dir = Path(path/'images/images')
artists = names['name'].str.replace(' ', '_').values

In [15]:
painting_list = []
for path, subdirs, files in os.walk(images_dir):
    for name in files:
        img = os.path.join(path, name)
        painting_list.extend([img])

#only works on windows
artist_list = []
for painting in painting_list:
    artist = painting.split('\\')[3]
    artist_list.extend([artist])

y = np.array(artist_list)  

encoder = LabelEncoder()
y = encoder.fit_transform(pd.Series(y))
classes = list(encoder.classes_)
for i in range(len(classes)):
    print(i, ' represents ', classes[i])
    
y = keras.utils.to_categorical(y, len(classes))
print(y)

X = np.array(painting_list) 

0  represents  Francisco_Goya
1  represents  Paul_Gauguin
2  represents  Pierre-Auguste_Renoir
3  represents  Rembrandt
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1)

In [17]:
batch_size = 32
training_batch_generator = CustomGenerator(X_train, y_train, batch_size)
validation_batch_generator = CustomGenerator(X_val, y_val, batch_size)
training_size = len(X_train)
validation_size = len(X_val)
test_size = len(X_test)

In [18]:
# Implement and Train A model based on VGG-16's architecture

# Generate the model
model = Sequential()

# Layer 1: Convolutional
model.add(Conv2D(input_shape=(224, 224, 3), filters=64, kernel_size=(3, 3),
                 padding='same', activation='relu'))

# Layer 2: Convolutional
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))

# Layer 3: MaxPooling
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 4: Convolutional
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 5: Convolutional
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 6: MaxPooling
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 7: Convolutional
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 8: Convolutional
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 9: Convolutional
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 10: MaxPooling
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 11: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 12: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 13: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 14: MaxPooling
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 15: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 16: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 17: Convolutional
model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
# Layer 18: MaxPooling
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

# Layer 19: Flatten
model.add(Flatten())
# Layer 20: Fully Connected Layer
model.add(Dense(units=4096, activation='relu'))
# Layer 21: Fully Connected Layer
model.add(Dense(units=4096, activation='relu'))
# Layer 22: Softmax Layer
model.add(Dense(units=len(classes), activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit_generator(generator=training_batch_generator,
                    steps_per_epoch=int(training_size // batch_size),
                    # should be 6 epochs
                    epochs=6,
                    verbose=1,
                    validation_data=validation_batch_generator,
                    validation_steps=int(validation_size // batch_size)
                   )

C:\Users\justu\AppData\Local\Temp\ipykernel_25292\273587099.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=training_batch_generator,


Epoch 1/6
30/30 [==============================] - 553s 18s/step - loss: 363.2348 - accuracy: 0.2812 - val_loss: 1.3033 - val_accuracy: 0.3854
Epoch 2/6
30/30 [==============================] - 540s 18s/step - loss: 1.3951 - accuracy: 0.3594 - val_loss: 1.2577 - val_accuracy: 0.3750
Epoch 3/6
30/30 [==============================] - 449s 15s/step - loss: 1.2408 - accuracy: 0.3927 - val_loss: 1.2450 - val_accuracy: 0.3854
Epoch 4/6
30/30 [==============================] - 278s 9s/step - loss: 1.1647 - accuracy: 0.4740 - val_loss: 1.2318 - val_accuracy: 0.3542
Epoch 5/6
30/30 [==============================] - 280s 9s/step - loss: 1.0663 - accuracy: 0.4896 - val_loss: 1.1073 - val_accuracy: 0.5208
Epoch 6/6
30/30 [==============================] - 264s 9s/step - loss: 1.1630 - accuracy: 0.4542 - val_loss: 1.1229 - val_accuracy: 0.4375


In [20]:
model.save(Path('model_smaller.h5'))